In [2]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np

Using TensorFlow backend.


In [7]:
def vgg19Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
 #   for layer in model.layers[140:]:
 #       layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model
    

In [4]:
import os
import shutil
from sklearn.utils import shuffle
np.random.seed(2017)

def prepare_val():
    dirname = 'imgs/val'
    dirname_src = 'imgs/train/'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for sub_dir in sub_dirs:
        address_des = dirname + '/' + sub_dir
        os.mkdir(address_des)
        
        address_src = dirname_src + '/' + sub_dir
        train_filenames = os.listdir(address_src)
        train_filenames = shuffle(train_filenames)
        for file_src in train_filenames[0: int(len(train_filenames) / 10)]:
            add_con_list = file_src.split('/')
            add_old = dirname_src + sub_dir + '/' + add_con_list[-1]
            add_new = dirname + '/' + sub_dir + '/' + add_con_list[-1]
         #   print(add_old)
         #   print(add_new)
            shutil.move(add_old, add_new)  
        
prepare_val()
    

In [14]:
image_size=(224, 224)
batch_size =64

In [9]:
#训练
nb_train_samples = 20197
def train_model(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
    
    train_generator = train_datagen.flow_from_directory(
    'imgs/train',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
    
    val_generator = train_datagen.flow_from_directory(
    'imgs/val',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
        
    model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=15,
    validation_data = val_generator)
    
    model.save_weights('model.h5')
    
    return model


In [12]:
import pandas as pd
import cv2
def test_model(model):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'
    test_filenames = os.listdir(test_address)
 #   test_filenames = test_filenames[:10]
    X = np.zeros((len(test_filenames), image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, test_file in enumerate(test_filenames) :
        X[i] = cv2.resize(cv2.imread(test_address + '/' + test_file), image_size)
    
    y_pred = model.predict(X, batch_size=batch_size, verbose=0)
  #  print(y_pred)
    
    for i, fname in enumerate(test_filenames):
        print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv('pred.csv', index=None)
    df.head(10)
    

In [ ]:
# all trainable true
model = train_model(vgg19Model)

Found 20187 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/15
315/315 [==============================] - 608s 2s/step - loss: 0.0458 - acc: 0.9862 - val_loss: 0.1460 - val_acc: 0.9598
Epoch 10/15
315/315 [==============================] - 608s 2s/step - loss: 0.0452 - acc: 0.9872 - val_loss: 0.1049 - val_acc: 0.9727
Epoch 11/15
315/315 [==============================] - 609s 2s/step - loss: 0.0432 - acc: 0.9873 - val_loss: 0.4226 - val_acc: 0.8882
Epoch 12/15
315/315 [==============================] - 609s 2s/step - loss: 0.0399 - acc: 0.9871 - val_loss: 0.9707 - val_acc: 0.8118
Epoch 13/15
315/315 [==============================] - 608s 2s/step - loss: 0.0436 - acc: 0.9871 - val_loss: 0.0959 - val_acc: 0.9678
Epoch 14/15
315/315 [==============================] - 606s 2s/step - loss: 0.0338 - acc: 0.9904 - val_loss: 0.0905 - val_acc: 0.9741
Epoch 15/15
315/315 [==============================] - 606s 2s/step - loss: 0.0294 - acc: 0.9911 - val_loss: 0

In [ ]:
test_model(model)

In [6]:
# all trainable false
train_model(vgg19Model)

94658560/94653016 [==============================] - 2s 0us/step
Found 20187 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/10
315/315 [==============================] - 343s 1s/step - loss: 1.5346 - acc: 0.4580 - val_loss: 2.5485 - val_acc: 0.3049
Epoch 2/10
315/315 [==============================] - 294s 933ms/step - loss: 0.9704 - acc: 0.6755 - val_loss: 2.1638 - val_acc: 0.3871
Epoch 3/10
315/315 [==============================] - 290s 922ms/step - loss: 0.8430 - acc: 0.7175 - val_loss: 2.5488 - val_acc: 0.3599
Epoch 4/10
315/315 [==============================] - 295s 935ms/step - loss: 0.8048 - acc: 0.7299 - val_loss: 3.2590 - val_acc: 0.2986
Epoch 5/10
315/315 [==============================] - 293s 930ms/step - loss: 0.7828 - acc: 0.7346 - val_loss: 2.5929 - val_acc: 0.3979
Epoch 6/10
315/315 [==============================] - 291s 925ms/step - loss: 0.7613 - acc: 0.7376 - val_loss: 3.0367 - val_acc: 0.3250
Epoch 7/10
315/315 [===============